In [1]:

import os
from dotenv import load_dotenv
import langchain, langchain_experimental,langchain_core,langchain_community, langchain_experimental
#loading the environment variables for API keys and also setting other env variable for longchain tracing
load_dotenv()
os.environ["LANGCHAIN_PROJECT"]="AgenticAI"
os.environ["LANGSMITH_TRACING"]="true"
os.environ["LANGCHAIN_TRACING_V2"]="true"

In [2]:
#Lets send q query to ChatGPT model
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model='gpt-4o')

## Simple query

In [3]:
result=llm.invoke("What is agentic AI")
result.content

'Agentic AI refers to artificial intelligence systems that are designed to operate with a degree of autonomy, making decisions and acting on behalf of their users or creators. These systems are typically capable of perceiving their environment, processing information, and performing actions to achieve specific goals or tasks. Agentic AI often involves the integration of machine learning, natural language processing, and other AI technologies to enable the system to perform complex tasks with minimal human intervention.\n\nThe concept of agentic AI contrasts with other forms of AI that are more passive or merely provide information without taking autonomous actions. Agentic AI systems might include robotic systems, autonomous vehicles, or virtual assistant software that can perform tasks ranging from scheduling meetings to performing complex logistics operations. The development and deployment of agentic AI involve careful consideration of ethical, security, and operational issues, as t

## Prompt Engineering using the LangChain

Prompt templates help to translate user input and parameters into instructions for a language model. This can be used to guide a model's response, helping it understand the context and generate relevant and coherent language-based output.

Prompt Templates take as input a dictionary, where each key represents a variable in the prompt template to fill in.

Prompt Templates output a PromptValue. This PromptValue can be passed to an LLM or a ChatModel, and can also be cast to a string or a list of messages. The reason this PromptValue exists is to make it easy to switch between strings and messages.

In [4]:
#These prompt templates are used to format a single string, and generally are used for simpler inputs. For example, a common way to construct and use a PromptTemplate is as follows:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("Tell me a joke about {topic}")
prompt_template.invoke({"topic": "cats"})



StringPromptValue(text='Tell me a joke about cats')

In [5]:
#These prompt templates are used to format a list of messages. These "templates" consist of a list of templates themselves. For example, a common way to construct and use a ChatPromptTemplate is as follows:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate([
    ("system", "You are a helpful assistant"),
    ("user", "Tell me a joke about {topic}")
])

prompt_template.invoke({"topic": "cats"})

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me a joke about cats', additional_kwargs={}, response_metadata={})])

In [6]:
#This prompt template is responsible for adding a list of messages in a particular place. In the above ChatPromptTemplate, we saw how we could format two messages, each one a string. 
# But what if we wanted the user to pass in a list of messages that we would slot into a particular spot? This is how you use MessagesPlaceholder.
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage

prompt_template = ChatPromptTemplate([
    ("system", "You are a helpful assistant"),
    MessagesPlaceholder("msgs")
])


prompt_template.invoke({"msgs": [HumanMessage(content="hi!")]})

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})])

In [7]:
#This will produce a list of two messages, the first one being a system message, and the second one being the HumanMessage we passed in. 
# If we had passed in 5 messages, then it would have produced 6 messages in total (the system message plus the 5 passed in). 
# This is useful for letting a list of messages be slotted into a particular spot.


#An alternative way to accomplish the same thing without using the MessagesPlaceholder class explicitly is:
prompt_template = ChatPromptTemplate([
    ("system", "You are a helpful assistant"),
    ("placeholder", "{msgs}") # <-- This is the changed part
])
prompt_template.invoke({"msgs": [HumanMessage(content="hi!")]})

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})])

## How use Prompts in LangChain

In [8]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_result=llm.invoke(messages)
ai_result

AIMessage(content="J'adore la programmation.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 31, 'total_tokens': 36, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8', 'id': 'chatcmpl-BdEZ1CfobyxBtYl3uyDzEzx0QwXc2', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--e6a21d39-13fb-4298-8f79-8f0888ed09d9-0', usage_metadata={'input_tokens': 31, 'output_tokens': 5, 'total_tokens': 36, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [9]:
#Chaining the models using prompts
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant that translates {input_language} to {output_language}."),
        ("human", "{input}"),
    ]
)

chain = prompt | llm
response=chain.invoke(
    {
        "input_language": "English",
        "output_language": "German",
        "input": "I love programming.",
    }
)
response.content

'Ich liebe das Programmieren.'

In [10]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")

chain = prompt | llm | StrOutputParser()
chain.invoke({"topic": "bears"})

'Why did the bear get kicked out of the campground?\n\nBecause he was always "paw-liticking"!'

In [11]:
## Json output formattor instead of the String?
from langchain_core.output_parsers import JsonOutputParser
promt_json= PromptTemplate(template="Answer the user queries. \n Tell me a joke about {topic} \n give ther response in {format}",
                                    input_variables=['topic'],
                                    partial_variables={'format': JsonOutputParser().get_format_instructions()}
                                    )

chain_json = promt_json | llm | JsonOutputParser()
chain_json.invoke({"topic": "cat"})

{'joke': 'Why was the cat sitting on the computer? Because it wanted to keep an eye on the mouse!'}

In [12]:
chatpromt_json= ChatPromptTemplate([("system","You are a stand up camedian /n and give response in {output_format}"),
                                    ('user',"tell me joke about {topic}")], 
                                    input_variables=['topic'],
                                    partial_variables={'output_format': JsonOutputParser().get_format_instructions()}
                                    )
chain_json = chatpromt_json | llm | JsonOutputParser()
chain_json.invoke({"topic": "dog"})

{'joke': "Why did the dog sit in the shade? Because he didn't want to be a hot dog!"}

In [17]:
## Using pydantic for the data type validation
product_promt=ChatPromptTemplate([('system','you are a product DB and returns top {num_products}, product ID, product name and Product price, returns in {format}'),
                                                ("user",'give 10 product list from product category type {product_category}')]
                                                )
product_chain= product_promt | llm | JsonOutputParser()
product_chain.invoke({"product_category":'Electornings',
                      "num_products":10,
                      "format":JsonOutputParser().get_format_instructions()})

[{'product_id': 'ELEC001',
  'product_name': 'Smartphone ProMax 12',
  'product_price': 999.99},
 {'product_id': 'ELEC002',
  'product_name': 'Ultra-HD 4K LED TV 55"',
  'product_price': 1499.99},
 {'product_id': 'ELEC003',
  'product_name': 'Noise-Canceling Headphones',
  'product_price': 349.95},
 {'product_id': 'ELEC004',
  'product_name': 'Laptop Core i7 16GB',
  'product_price': 1199.89},
 {'product_id': 'ELEC005',
  'product_name': 'Wireless Bluetooth Speaker',
  'product_price': 129.99},
 {'product_id': 'ELEC006',
  'product_name': 'Smartwatch Series 6',
  'product_price': 399.99},
 {'product_id': 'ELEC007',
  'product_name': 'Gaming Console Pro',
  'product_price': 499.99},
 {'product_id': 'ELEC008',
  'product_name': 'Tablet 10.5" 128GB',
  'product_price': 599.99},
 {'product_id': 'ELEC009',
  'product_name': 'Digital Camera 24MP',
  'product_price': 799.95},
 {'product_id': 'ELEC010',
  'product_name': 'Home Assistant Smart Hub',
  'product_price': 199.99}]

In [23]:
## Validating ouput with Pydantic

from pydantic import BaseModel
class Product(BaseModel):
    product_id: str
    product_description: str
    product_price: float

json_parser= JsonOutputParser(pydantic_object=Product)

product_chain= product_promt | llm | json_parser
product_chain.invoke({"product_category":'Electronincs',
                      "num_products":10,
                      "format":json_parser.get_format_instructions()})

[{'product_id': 'E1001',
  'product_description': 'Smartphone XYZ Pro',
  'product_price': 999.99},
 {'product_id': 'E1002',
  'product_description': '4K Ultra HD Smart TV 55in',
  'product_price': 549.99},
 {'product_id': 'E1003',
  'product_description': 'Bluetooth Noise Cancelling Headphones',
  'product_price': 199.99},
 {'product_id': 'E1004',
  'product_description': 'Laptop Pro 16GB RAM, 512GB SSD',
  'product_price': 1299.99},
 {'product_id': 'E1005',
  'product_description': 'Smartwatch Series 6',
  'product_price': 399.99},
 {'product_id': 'E1006',
  'product_description': 'Wireless Home Security Camera',
  'product_price': 89.99},
 {'product_id': 'E1007',
  'product_description': 'Portable Bluetooth Speaker',
  'product_price': 49.99},
 {'product_id': 'E1008',
  'product_description': 'Gaming Console X',
  'product_price': 499.99},
 {'product_id': 'E1009',
  'product_description': 'Digital SLR Camera 24MP',
  'product_price': 749.99},
 {'product_id': 'E1010',
  'product_desc

### Coersion
We can even combine this chain with more runnables to create another chain. This may involve some input/output formatting using other types of runnables, depending on the required inputs and outputs of the chain components..-


In [ ]:
#For example, let's say we wanted to compose the joke generating chain with another chain that evaluates whether or not the generated joke was funny.
analysis_prompt = ChatPromptTemplate.from_template("is this a funny joke? {joke}")
composed_chain = {"joke": chain} | analysis_prompt | llm | StrOutputParser()
composed_chain.invoke({"topic": "bears"})

'Humor is subjective, so what one person finds funny, another might not. The joke you provided is a play on words, using "bear feet" to humorously sound like "bare feet," which is a common trait of bears. People who enjoy puns might find this joke amusing due to the clever wordplay.'

Note: The information here refers to parsers that take a text output from a model try to parse it into a more structured representation. More and more models are supporting function (or tool) calling, which handles this automatically. It is recommended to use function/tool calling rather than output parsing. 

## Tool Calling
#not required in this

In [ ]:
from pydantic import BaseModel, Field

#String Validation
class GetWeather(BaseModel):
    """Get the current weather in a given location"""
    location: str = Field(..., description="The city and state, e.g. San Francisco, CA")


llm_with_tools = llm.bind_tools([GetWeather])
weather = llm_with_tools.invoke("what is the weather like in San Francisco")
weather.content

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_yCYluuHss61Qk1XRiabIfwxS', 'function': {'arguments': '{"location":"San Francisco, CA"}', 'name': 'GetWeather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 68, 'total_tokens': 85, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8', 'id': 'chatcmpl-BdDdVTHHuwiHtKbCzdJ2BpumqivUq', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--cbd462d9-da97-4070-a84a-6c6d782a298a-0', tool_calls=[{'name': 'GetWeather', 'args': {'location': 'San Francisco, CA'}, 'id': 'call_yCYluuHss61Qk1XRiabIfwxS', 'type': 'tool_call'}], usage_metadata={'input_tokens': 68, 'output_tokens': 17, 'total_tokens': 85, 'input